# Advanced Lane Finding


## Camera Calibration

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import os

#Function which calibrates the camera
def calibrateCamera():
    # Generate object points for chess board
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)
    #print (np.mgrid[0:9, 0:6])
    
    # Obj points and source points for all images
    ObjPoints = []
    SrcPoints = []

    # Look for calibration images
    calib_images = glob.glob("camera_cal/calibration*.jpg")

    for calib_image in calib_images:
        img = cv2.imread(calib_image)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_size = gray_img.shape[::-1]

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray_img, (9, 6), None)

        if (ret==True):
            ObjPoints.append(objp)
            SrcPoints.append(corners)

            # Draw and display the corners
            #cv2.drawChessboardCorners(img, (9,6), corners, ret)
            #cv2.imwrite("output_images/" + calib_image, img)
            
    #Find the calibration parameters for the camera given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(ObjPoints, SrcPoints, img_size, None, None)
    
    return mtx, dist
    
# Returns undistorted image given the camera matrix and distortion coefficients
def undistortImage(img, mtx, dist):
    undistImg = cv2.undistort(img, mtx, dist, None, mtx)
    return undistImg

# Calculates the perspective transform. The coordinates are taken from test_images/straight_lines1.jpg
def calculatePerspectiveTransform():
    #src = np.float32([[610, 439], [670, 439], [1060, 679], [248, 679]])
    #src = np.float32([[588, 452], [694, 452], [1060, 679], [248, 679]])
    src = np.float32([[600, 445], [680, 445], [1060, 679], [248, 679]])
    dst = np.float32([[200, 0], [930, 0], [930, 719], [200, 719]]) # Assymmetric rectangle
    M = cv2.getPerspectiveTransform(src, dst)
    return M

# Calculates the inverse perspective transform. The coordinates are taken from test_images/straight_lines1.jpg
# During inverse perspective tramsform, the car bonnet area is also included to get extended lane
def calculateInversePerspectiveTransform():
    # points calculated from straight_lines1.jpg
    #src = np.float32([[610, 439], [670, 439], [1060, 679], [248, 679]])
    #src = np.float32([[610, 439], [670, 439], [1110, 719], [206, 719]])
    #src = np.float32([[588, 452], [694, 452], [1110, 719], [206, 719]])
    src = np.float32([[600, 445], [680, 445], [1110, 719], [206, 719]])
    dst = np.float32([[200, 0], [930, 0], [930, 719], [200, 719]]) # Assymmetric rectangle
    M = cv2.getPerspectiveTransform(dst, src)
    return M

# Does the perspective Transform for the given image 
def doPerspectiveTransform(bin_img, M):
    warped_bin_img = cv2.warpPerspective(bin_img, M, bin_img.shape[1::-1], flags=cv2.INTER_LINEAR)
    return warped_bin_img

# Does the perspective Transform for the given array
def doPerspectiveTransformPoints(srcArray, M):
    dstArray = cv2.perspectiveTransform(srcArray, M)
    return dstArray

In [2]:

# Calculates the sobel threshold 
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    # Apply threshold
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    # 3) Take the absolute value of the derivative or gradient
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    # 6) Return this mask as your binary_output image
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if(orient == 'x'):
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    elif(orient == 'y'):
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
        
    abs_sobel = np.absolute(sobel)
    
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    sobel_binary = np.zeros_like(scaled_sobel)
    sobel_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1 
    return sobel_binary

# Calculates the magnitude gradient
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    # Apply threshold
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

# Find the directional threshold
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

# Find the HLS color threshold
def hls_s_threshold(img, s_thresh=(0, 255)):
    # Convert to HLS color space and separate the S channel
    # Note: img is the undistorted image
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    s_channel = hls[:,:,2]
    
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Return the binary image
    return s_binary

In [3]:

# Returns the histogram of the image
def hist(warped_img):
    # Grab only the bottom half of the image
    # Lane lines are likely to be mostly vertical nearest to the car
    bottom_half = warped_img[warped_img.shape[0]//2:,:]

    # Sum across image pixels vertically - make sure to set `axis`
    # i.e. the highest areas of vertical lines should be larger values
    histogram = np.sum(bottom_half, axis=0)
    
    return histogram

# Find the lane pixel positions by searching using sliding window approach
# Returns the leftx, lefty, rightx, righty, out_img()
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

# Returns the left_fit, right_fit polynomial coefficients by sliding window approach
def fit_polynomial_sliding_window(binary_warped):
    # Find our lane pixels first by sliding window approach
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')

    return left_fit, right_fit, out_img

def fit_poly(img_shape, leftx, lefty, rightx, righty):
     ### Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fit, right_fit, left_fitx, right_fitx, ploty

# returns a new polynomial coefficients (left_fit, right_fit) using the left_fit and right_fit of previous image
def search_around_poly(binary_warped, left_fit, right_fit):
    # HYPERPARAMETER
    margin = 50

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fit, right_fit, left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    right_line_window1 = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    
    window_pts = np.hstack((left_line_window1, right_line_window1))
    
    cv2.fillPoly(window_img, np.int_([window_pts]), (0,255, 0))
    
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return left_fit, right_fit, result

# Does the reverse transform of the lane polygon and returns the image overlayed polygon on it
def doReverseTransform(img, left_fit, right_fit, M):
    #if(left_fitx.size == 0 | right_fitx.size == 0 | ploty.size == 0):
    #    return img
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    ### Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Reverse Transform the left lane points
    left_points = np.dstack([left_fitx, ploty])   
    left_points_transformed = doPerspectiveTransformPoints(left_points, M)
    
    # Reverse Transform the right lane points
    right_points = np.dstack([right_fitx, ploty])
    right_points_transformed = doPerspectiveTransformPoints(right_points, M)
    
    # Stack the left and right transformed points
    window_pts = np.hstack((left_points_transformed, right_points_transformed[:,::-1,:]))
    
    # Draw the polygon
    window_img = np.zeros_like(img)
    cv2.fillPoly(window_img, np.int_([window_pts]), (0,255, 0))
    
    # Find the resultant image
    result_img = cv2.addWeighted(img, 1, window_img, 0.3, 0)
    
    return result_img

# Calculates the left and right lane curvature
def measureCurvatureReal(imgShape, left_fit, right_fit):
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 50/720# meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, imgShape[0]-1, imgShape[0])
    
    left_fitx = left_fit[0] * np.square(ploty) + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * np.square(ploty) + right_fit[1] * ploty + right_fit[2]
    
    #ym_per_pix = 1/1000
    #xm_per_pix = 1/1000
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    
    # Evaluate the curvature at 50% of max y
    y_eval = np.max(ploty) * 0.50
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

# Returns the position of the car in metres from the left lane
def findPositionOfCar(imgShape, left_fit, right_fit):
    
    #Find the left point and right point at bottom of the y axis
    y = imgShape[0] - 1
    left_x = left_fit[0] * np.square(y) + left_fit[1] * y + left_fit[2]
    right_x = right_fit[0] * np.square(y) + right_fit[1] * y + right_fit[2]
    
    #left_right_points = np.array([[left_x, y], [right_x, y]])
    
    #transformed_points_left = doPerspectiveTransformPoints(np.dstack([left_x, y]), M)
    #transformed_points_right = doPerspectiveTransformPoints(np.dstack([right_x, y]), M)
    
    #carDistanceInPixels = (transformed_points_left[0] + transformed_points_right[0])/2 - imgShape[1]/2
    #carDistanceInMeters = 3.7/(transformed_points_right[0] - transformed_points_left[0]) * carDistanceInPixels
    
    #Used 1130 as image width to account for assymetric rectangle used in perspective transformation
    carDistanceInPixels = (left_x + right_x)/2 - 1130/2
    carDistanceInMeters = 3.7/(right_x - left_x) * carDistanceInPixels
    
    return carDistanceInMeters
    #print("Distance in pixels: ", carDistanceInPixels)
    #print("Distance in m: ", 3.7/700 * carDistanceInPixels)

# Does the sanity check for for left and right lane coefficients
def doLaneSanityCheck(imgShape, left_fit, right_fit):
    # Whether the lanes are valid or not
    valid = True
    
    #Find the distance between lanes at the bottom point
    bottom_y = imgShape[0] - 1
    left_bot_x = left_fit[0] * np.square(bottom_y) + left_fit[1] * bottom_y + left_fit[2]
    right_bot_x = right_fit[0] * np.square(bottom_y) + right_fit[1] * bottom_y + right_fit[2]
    bottomDistanceInPixels = right_bot_x - left_bot_x
    
    #Find the distance between lanes at the top point
    top_y = 0
    left_top_x = left_fit[0] * np.square(top_y) + left_fit[1] * top_y + left_fit[2]
    right_top_x = right_fit[0] * np.square(top_y) + right_fit[1] * top_y + right_fit[2]
    topDistanceInPixels = right_top_x - left_top_x
    
    #Find the diff distance
    diffDistanceInPixels = np.abs(bottomDistanceInPixels - topDistanceInPixels)
    
    #print(' bottomDistanceInPixels ', bottomDistanceInPixels.astype('int'))
    #print(' topDistanceInPixels ', topDistanceInPixels.astype('int'))
    #print(' diffDistance ', diffDistance.astype('int'))
    
    # Do some validity checks for bottom distance
    if( (600 > bottomDistanceInPixels.astype('int')) | (bottomDistanceInPixels.astype('int') > 1100)):
        valid = False
        
    # Do some validity checks for top distance
    if( (500 > topDistanceInPixels.astype('int')) | (topDistanceInPixels.astype('int') > 1200)):
        valid = False
        
    # If the difference distance between bottom and top line is more than 400 pixels, then it is invalid
    if(diffDistanceInPixels.astype('int') > 400):
        valid = False
    
    return valid, diffDistanceInPixels
    
    
    

In [4]:
#-------------------------------------------------------
#Actual Code
# Class which does advanced lane finding
class LaneFinding():
    def __init__(self):
        # Boolean which defines the poly to be found using the sliding window approach
        self.first_image = False
        # Last left fit polynomials
        self.left_fit = np.array(1*3)
        # Last right fit polynomials
        self.right_fit = np.array(1*3)
        # Finds the calibration parameters
        self.mtx, self.dist = calibrateCamera()
        # Calculates the perspective tranformation matrix
        self.M = calculatePerspectiveTransform()
        # Calculates the inverse perspective tranformation matrix
        self.inverse_M = calculateInversePerspectiveTransform()
        
        self.saveImage = False
        
    # Saves the calibration images into output_images folder
    def saveCalibImages(self):
        calib_images = glob.glob("camera_cal/calibration*.jpg")
        for calib_image in calib_images:
            img = cv2.imread(calib_image)
            undistImg = undistortImage(img, self.mtx, self.dist)
            cv2.imwrite("output_images/" + calib_image, undistImg)
            
    # Method which process a single image and returns the lane overlayed image
    def process_image(self, img):
        undistImg = undistortImage(img, self.mtx, self.dist)
        #cv2.imwrite("output_images/undistorted_" + file, undistImg)

        # Choose a Sobel kernel size
        ksize = 3 # Choose a larger odd number to smooth gradient measurements

        # Apply each of the thresholding functions
        gradx = abs_sobel_thresh(undistImg, orient='x', sobel_kernel=ksize, thresh=(20, 100))
        grady = abs_sobel_thresh(undistImg, orient='y', sobel_kernel=ksize, thresh=(40, 100))
        mag_binary = mag_thresh(undistImg, sobel_kernel=ksize, mag_thresh=(50, 100))
        dir_binary = dir_threshold(undistImg, sobel_kernel=5, thresh=(0.7, 1.3))

        s_binary = hls_s_threshold(undistImg, s_thresh=(150, 255))

        #combined_grad = np.zeros_like(dir_binary)
        combined = np.zeros_like(dir_binary)
        combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1)) | ((s_binary == 1))] = 1    

        #cv2.imwrite("output_images/gradx_" + file, gradx * 255)
        #cv2.imwrite("output_images/grady_" + file, grady * 255)
        #cv2.imwrite("output_images/mag_binary_" + file, mag_binary * 255)
        #cv2.imwrite("output_images/dir_binary_" + file, dir_binary * 255)
        #cv2.imwrite("output_images/combined_" + file, combined * 255)
        #cv2.imwrite("output_images/hls_binary_" + file, s_binary * 255)

        #Perspective Transform
        warped_bin_image = doPerspectiveTransform(combined, self.M)

        #cv2.imwrite("output_images/warped_combined_" + file, warped_bin_image * 255)

        #histogram = hist(warped_bin_image)
        # Visualize the resulting histogram
        #plt.plot(histogram)

        # calculate fit polynomial by the sliding window approach for the first image
        if(self.first_image == False):  
            self.first_image = True
            self.left_fit, self.right_fit, poly_fitted_image = fit_polynomial_sliding_window(warped_bin_image)
            

        # Search using the last lane polynomial parameters
        left_fit, right_fit, poly_fitted_image = search_around_poly(warped_bin_image, self.left_fit, self.right_fit)
        
        #if(self.saveImage == False):
        #    self.saveImage = True
        #    cv2.imwrite("output_images/lane_curve_detected_image.jpg", poly_fitted_image)
        
        # Do the lane sanity check
        valid, diffDistance = doLaneSanityCheck(poly_fitted_image.shape, left_fit, right_fit)
        
        # If valid copy the parameters, otherwise search once again using sliding window approach
        if(valid == True):
            self.left_fit, self.right_fit = left_fit, right_fit
        else:
            self.left_fit, self.right_fit, poly_fitted_image = fit_polynomial_sliding_window(warped_bin_image)

        #return poly_fitted_image #warped_bin_image * 255
        
        # Draw the lane polygon as a overlay onto the image
        final_img = doReverseTransform(img, self.left_fit, self.right_fit, self.inverse_M)
        
        # Measure the curvatures
        left_curverad, right_curverad = measureCurvatureReal(poly_fitted_image.shape, self.left_fit, self.right_fit)
        
        #Find the car distance in meters
        carDistanceInMeters = findPositionOfCar(poly_fitted_image.shape, self.left_fit, self.right_fit)
        
        # Display the details
        cv2.putText(final_img, 'Curvature: L: ' + str(left_curverad) + '(m) R: ' + str(right_curverad) + '(m)', (10,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2, cv2.LINE_AA)
        cv2.putText(final_img, 'Vehicle Centre is at: ' + str(carDistanceInMeters) + '(m) from lane centre', (10,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2, cv2.LINE_AA)
        #cv2.putText(final_img, 'valid: ' + str(valid) + ' diffDistance: ' + str(diffDistance), (10,300), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2, cv2.LINE_AA)
        
        return final_img
        

In [5]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

laneFindingObj = LaneFinding()
laneFindingObj.saveCalibImages()

video_output = 'project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('project_video.mp4')#.subclip(25, 35)
yellow_clip = clip2.fl_image(laneFindingObj.process_image)
%time yellow_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████▉| 1260/1261 [07:07<00:00,  2.94it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

CPU times: user 4min 15s, sys: 39.1 s, total: 4min 54s
Wall time: 7min 10s


In [6]:
#test_images = os.listdir("test_images/")

#for file in test_images:
#    print("file:--- ", file)
#    file_fullpath = "test_images/" + file
#    img = cv2.imread(file_fullpath)
#    process_image(img)